In [ ]:
from __future__ import absolute_import, division, print_function
import numpy as np
import keras
import matplotlib.pyplot as plt
import ast
import numpy as np
import sys
import talos as ta
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, Nadam
from keras.activations import relu, elu, tanh, sigmoid, softmax
from keras.losses import sparse_categorical_crossentropy

In [ ]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
typeV = "deadlock"

In [ ]:
# LOAD A MODEL IF YOU HAVE ONE
#from keras.models import load_model
#model = load_model('/obfuscated/VULN/'+typeV+'/'+typeV+'.h5')

In [ ]:
#Read change files
fin = open('obfuscated/vulnerable_methods_VNV.txt', 'r')
unique_operations = []
for line in fin:
    line = line.strip()
    operations = line.split(' ')
    for operation in operations:
        operation = operation.strip()
        if operation not in unique_operations:
            unique_operations.append(operation)

fin.close()
print(unique_operations)

In [ ]:
#Store the indices
fout = open('/obfuscated/'+typeV+'/index_'+typeV+'.csv', 'w')
for operation in unique_operations:
    fout.write('%d,\"%s\"\n' % (unique_operations.index(operation),operation))
fout.close()

In [ ]:
#Read the indices
mapping = dict()
fin = open('obfuscated/'+typeV+'/index_'+typeV+'.csv', 'r')
for line in fin:
    line = line.strip()
    index_separator = line.index(',"')
    index = int(line[:index_separator])
    operation = line[index_separator+2:len(line)-1]
    mapping[operation] = index
fin.close()
mapping['<PAD>'] = (len(mapping))
print(mapping)

In [ ]:
# Convert the strings to vectors of numbers
fin = open('/obfuscated/VULN/'+typeV+'/vulnerable_methods_'+typeV+'.txt', 'r')
fout = open('/obfuscated/VULN/'+typeV+'/vulnerable_methods_numbers_'+typeV+'.txt', 'w')
for line in fin:
    line = line.strip()
    operations = line.split(' ')
    to_print = []
    for operation in operations:
        operation = operation.strip()
        to_print.append(mapping[operation])
    fout.write('%s\n' % to_print)
fin.close()
fout.close()

In [ ]:
#Read the dataset as a tuple of lists
dataset = list()
fin = open('/obfuscated/VULN/'+typeV+'/vulnerable_methods_numbers_'+typeV+'.txt', 'r')
count = 0
for line in fin:
    count += 1
    dataset.append(np.array(ast.literal_eval(line)))
    #if(count > 19999): #TO FIX
     #   break
fin.close()
print(str(len(dataset)))

In [ ]:
#Create labels set, and convert it to numbers
fin = open('/obfuscated/VULN/'+typeV+'/matching_tags_'+typeV+'.txt', 'r')
fout = open('/obfuscated/VULN/'+typeV+'/matching_tags_numbers_'+typeV+'.txt', 'w')
count = 0
unique_tags = dict()
vulnerabilities = []
for line in fin:
    line = line.strip()
    unique_tags[line] = unique_tags.get(line, 0) + 1
    if line not in vulnerabilities:
        vulnerabilities.append(line)
    fout.write('%d\n' % vulnerabilities.index(line))
fin.close()
fout.close()

print(unique_tags)
print("-----------------------")
print(vulnerabilities)
print("-----------------------")
print(len(vulnerabilities),'vulnerability types')

In [ ]:
#Read the label set as numpy array
labels = list()
fin = open('/obfuscated/'+typeV+'/matching_tags_numbers_'+typeV+'.txt', 'r')
count = 0
for line in fin:
    line = line.strip()
    count += 1
    labels.append(int(line))
    #if(count > 19999): #TO FIX
     #   break
labels = np.array(labels)
fin.close()
print(str(len(labels)))

In [ ]:
#Normalize the length (number of operations) of the commits
dataset = keras.preprocessing.sequence.pad_sequences(dataset,
                                                     value=mapping["<PAD>"], #Sequences that are shorter than maxlen are padded with value at the end.
                                                     padding='post') #If not provided, is the value of the longest sequence


In [ ]:
#Create the datasets
training_percentage = 0.8
validation_percentage = 0.1
test_percentage = 0.1

training_size = int(len(dataset)*training_percentage)
validation_size = int(len(dataset)*validation_percentage)
test_size = int(len(dataset)*test_percentage)

training_data = dataset[:training_size]
training_labels = labels[:training_size]

validation_data = dataset[training_size:training_size+validation_size]
validation_labels = labels[training_size:training_size+validation_size]

test_data = dataset[training_size+validation_size:]
test_labels = labels[training_size+validation_size:]

# vocabulary size
vocab_size = len(mapping)
print(vocab_size)

In [ ]:
#Build the model
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 32)) #Embedding layer: takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. These vectors are learned as the model trains.
#see how to save the generated embeddings here: https://www.tensorflow.org/alpha/tutorials/sequences/word_embeddings#retrieve_the_learned_embeddings
model.add(keras.layers.GlobalAveragePooling1D()) #returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model to handle input of variable length, in the simplest way possible.
model.add(Dense(60, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

#train the model (pay attention to #epochs, it strongly influences the execution time)
history = model.fit(training_data,
                    training_labels,
                    epochs=2000,
                    batch_size=46,
                    validation_data=(validation_data, validation_labels), #monitors the accuracy on the validation set
                    verbose=1,
                    shuffle = True)

In [ ]:
#evaluate the model on test data
results = model.evaluate(test_data, test_labels)
predictions = model.predict_classes(test_data)
print(results)
print(predictions)

In [ ]:
fout = open('/obfuscated/'+typeV+'/predictions.csv', 'w')
fout.write('actual,predicted,correct\n')
for i in range(len(predictions)):
    fout.write('%d,%d,%r\n' % (test_labels[i],predictions[i][0],test_labels[i] == predictions[i][0]))
fout.close()

In [ ]:
#plot what happened during the training
history_dict = history.history #model.fit() returns a History object that contains a dictionary with everything that happened during training
history_dict.keys()

acc = history_dict['acc']
val_acc = history_dict['val_acc']
loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = range(1, len(acc) + 1)

#training and validation accuracy
plt.figure(figsize=(10,10))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
yhat_classes = model.predict_classes(test_data, verbose=1)

In [ ]:
print(yhat_classes)

In [ ]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_labels, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
print("---------")
precision = precision_score(test_labels, yhat_classes, average=None)
print(precision)
precision = precision_score(test_labels, yhat_classes, average='micro')
print('Precision: %f' % precision)
print("---------")
# recall: tp / (tp + fn)
recall = recall_score(test_labels, yhat_classes, average=None)
print(recall)
recall = recall_score(test_labels, yhat_classes, average='micro')
print('Recall: %f' % recall)
print("---------")
# # f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test_labels, yhat_classes, average=None)
print(f1)
f1 = f1_score(test_labels, yhat_classes, average='micro')
print('F1 score: %f' % f1)

In [ ]:
confusion_matrix(test_labels, yhat_classes)

In [ ]:
#target_names = ['deadlock', 'infinite loop', 'null pointer dereference', 'race condition', 'double free', 'dead code', 'buffer overflow', 'use after free']
target_names = ['VULN', 'NOVULN']
print(classification_report(test_labels, yhat_classes, target_names=target_names))

In [ ]:
model.save('obfuscated/'+typeV+'/'+typeV+'.h5')